# Tarea 1

Implementar la clase `OisCashflow`.

## Configuración Inicial

In [15]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum
import pandas as pd

In [2]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [167]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2019, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(11, 11, 2019))
        cal.add_holiday(Qcf.QCDate(28, 11, 2019))
        cal.add_holiday(Qcf.QCDate(25, 12, 2019))
        cal.add_holiday(Qcf.QCDate(20, 1, 2020))
        cal.add_holiday(Qcf.QCDate(17, 2, 2020))
        cal.add_holiday(Qcf.QCDate(10, 4, 2020))
        cal.add_holiday(Qcf.QCDate(25, 5, 2020))
        cal.add_holiday(Qcf.QCDate(3, 7, 2020))
        cal.add_holiday(Qcf.QCDate(7, 9, 2020))
        cal.add_holiday(Qcf.QCDate(30, 9, 2020))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
        
    return cal

## `Qcf.time_series`

In [4]:
ts = Qcf.time_series()

fecha1 = Qcf.QCDate(13, 1, 1969)
ts[fecha1] = 19690113

fecha2 = Qcf.QCDate(14, 1, 1969)
ts[fecha2] = 19690114

print(f'ts[fecha1]: {ts[fecha1]}\n')

dl = Qcf.time_series_dates(ts)
for d in dl:
    print(d)
print()
    
vl = Qcf.time_series_values(ts)
for v in vl:
    print(v)

ts[fecha1]: 19690113.0

13-1-1969
14-1-1969

19690113.0
19690114.0


### Se construye a partir de Data

In [28]:
df_fixings = pd.read_excel('data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [48]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [30]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [54]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

## Clase `OisCashflow`

La clase `OisCashflow` es una [`dataclass`](https://realpython.com/python-data-classes/). La tarea consiste en:

- Implementar los 3 métodos de la clase que están con `pass`.
- Implementar la función `present_value`.
- Implementar la función `set_expected_rate`.

In [184]:
@dataclass  # syntactic sugar
class OisCashflow:
    start_date: Qcf.QCDate
    end_date: Qcf.QCDate
    settlement_date: Qcf.QCDate
    notional: float
    currency: Qcf.QCCurrency
    amortization: float
    amort_is_cashflow: bool
    interest_rate: Qcf.QCInterestRate
    on_index: Qcf.InterestRateIndex
    spread: float
    gearing: float
    calendar: Qcf.BusinessCalendar # tuve que agregar esta variable

    def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula la tasa equivalente desde `self.start_date` a `accrual_date`. La tasa equivalente
        se calcula como:

        (P - 1) * 360 / (accrual_date - self.start_date)

        donde P es el producto de los factores de capitalización de todas las tasas overnight
        entre `start_date` y `accrual_date`. Los valores de esas tasas deben estar almacenados
        en la variable `fixings`.

        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        if self.calendar.next_busy_day(accrual_date) > accrual_date:
            raise ValueError("Acrrual date must be a working day.")
        fecha = self.start_date
        factor = 1.0
        while fecha < accrual_date:
            fecha_siguiente = self.calendar.shift(fecha, 1)
            tasa = self.on_index.get_rate()
            try:
                tasa.set_value(fixings[fecha])
            except Exception as e:
                raise ValueError(f'Se produjo el error: {str(e)}. Fecha de cálculo: {fecha.description(False)}')
            factor *= tasa.wf(fecha, fecha_siguiente)
            fecha = fecha_siguiente
        return self.interest_rate.get_rate_from_wf(factor, self.start_date, accrual_date)

    def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula el interés (plata) devengado desde `self.start_date` a `accrual_date` utilizando la tasa equivalente
        que se calcula con el método anterior.

        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.

        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        valor_tasa = self.get_accrued_rate(accrual_date, fixings)
        self.interest_rate.set_value(valor_tasa)
        return self.notional * (self.interest_rate.wf(self.start_date, accrual_date) - 1.0)

    def amount(self, fixings: Qcf.time_series) -> float:
        """
        Calcula el flujo total al vencimiento (amortización más intereses devengados hasta self.end_date).

        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.

        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        return self.get_accrued_interest(self.end_date, fixings) + self.notional

Construcción de un objeto `Qcf.InterestRateIndex`.

In [185]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)

Construcción de una instancia de `OisCashflow`.

In [186]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1,
    settlement_calendar
)

In [188]:
print(f'accrued rate: {ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings):.8%}')
print(f'accrued interest: {ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings):,.2f}')
print(f'amount: {ois.amount(fixings):,.2f}')

accrued rate: 1.04766527%
accrued interest: 75,082.68
amount: 10,077,848.10


## Funciones

In [11]:
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois_cashflow: OisCashflow,
        zcc: Qcf.ZeroCouponCurve,
        fixings: Qcf.time_series) -> None:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Ver por ejemplo los casos cuando usamos el objeto Qcf.ForwardRates() en
    el notebook 9.
    
    Para un OIS calculando en t0 un cashflow entre (t0 <) t1 < t2
    ICP(t0)*wf(to, t2) = ICP(t2), ICP(t0)*w(t0, t1)=ICP(t1) -> ICP(t2)/ICP(t1)=ICP(t1,t2)
    tasa(t1, t2) = [ICP(t1, t2) - 1]*360 / (t2-t1)
    El ICP se puede pensar como el account asociado a la tasa interbancaria chilena, que es el índice
    overnight chileno.
    
    Cuando t1 < t0 < t2 tener un poquito de ojo ... porque el cashflow ya tiene fixings.
    """
    pass


In [12]:
def present_value(val_date: Qcf.QCDate,
                  ois_cashflow: OisCashflow,
                  zcc: Qcf.ZeroCouponCurve) -> float:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Para probar esta función utilizar la data de que está en data/20201012_built_sofr_zero.xlsx.
    Para traer a valor presente debes respetar la convención de las tasas de la curva.
    
    NOTA: la variable ois_cashflow debe haber 'pasado' por la función set_expected_rate para que
    tenga los intereses de los fixings futuros.
    """
    pass